<a href="https://colab.research.google.com/github/Fran-Sierra/Data-analysis-in-Football/blob/main/colab_files/Plot_attack_%26_Pass_nodes_V_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install mplsoccer

In [4]:
from google.colab import auth
from mplsoccer import Pitch, Sbopen, VerticalPitch, FontManager, FontManager, add_image,Sbopen
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import to_rgba
from urllib.request import urlopen
from PIL import Image


# Will collect your credentials
auth.authenticate_user()

#varglobals // solo cambia a la direccion de tu proyecto
project_id="formal-plating-405021"

In [5]:
#Consultar Competicion
def competitions():
  query=f'''
    select *
    from `{project_id}.consultor_dt.competitions`
  '''
  df_competitions = pd.read_gbq(query, project_id)
  return df_competitions

In [7]:
#consultar equipo
def teams():
  query=f'''
    select *
    from `{project_id}.consultor_dt.teams`
  '''
  df_teams = pd.read_gbq(query, project_id)
  return df_teams


In [10]:
#consultar matches_countryName
def matches_country(countryName):
    query_cntry=f'''
    select matches.*,
    teams.officialName as home_name,
    teams2.officialName as away_name
    from `{project_id}.consultor_dt.matches_{countryName.lower()}` as matches
    left join `{project_id}.consultor_dt.teams` as teams on teams.teamId=matches.home_team
    left join `{project_id}.consultor_dt.teams` as teams2 on teams2.teamId=matches.away_team
    order by gameweek
    '''

    df_matches = pd.read_gbq(query_cntry, project_id)
    return df_matches

In [13]:
# Función para obtener los partidos en los que el equipo participó
def get_team_matches_df(df_matches, team_id):
    team_matches = df_matches[(df_matches['home_team'] == team_id) | (df_matches['away_team'] == team_id)]
    return team_matches

In [14]:
# Verifica si el match_id ingresado es válido para el equipo seleccionado
def match_is_valid(match_id, matches_df, teams_df):
    match_info = matches_df[matches_df['matchId'] == match_id]
    if not match_info.empty:
        home_team = match_info['home_team'].iloc[0]
        home_team_name = teams_df[teams_df['teamId'] == home_team]['club_name'].iloc[0]
        away_team = match_info['away_team'].iloc[0]
        away_team_name = teams_df[teams_df['teamId'] == away_team]['club_name'].iloc[0]
        print(f"El partido fue: {home_team_name} vs {away_team_name}")

Ingresar la Informacion

In [15]:
def input_info():
    # Imprime las competiciones disponibles
    print(competitions()[['competition_name', 'competitionId']])

    # Inicializa las variables
    country_name_df = competitions()
    teams_df = teams()

    # Utilizamos un bucle para asegurarnos de que se ingrese un competition_id válido
    while True:
        try:
            competition_id = int(input('Introduce competition_id: '))

            # Verifica si el competition_id ingresado es válido
            if competition_id in competitions()['competitionId'].values:
                # Imprime el nombre del país para la competición seleccionada
                country_name = country_name_df[country_name_df['competitionId'] == competition_id]['counrty'].unique()[0].lower()
                print(f'País asociado a competition_id {competition_id}: {country_name}')
                break
            else:
                print(f'El competition_id {competition_id} no está en la base o no es válido. Por favor, introduce un competition_id válido.')
        except ValueError:
            print('Por favor, introduce un valor válido.')

    # Creamos la variable df_matches_country enviando el nombre del país seleccionado
    df_matches_country = matches_country(country_name)

    # Almacena solo los equipos locales únicos
    unique_home_teams = df_matches_country['home_team'].unique()

    # Utilizamos un bucle para asegurarnos de que se ingrese un team_id válido
    while True:
        try:
            team_id = int(input('Introduce team_id: '))
            # Verifica si el team_id ingresado es válido para la competición seleccionada
            if team_id in unique_home_teams:
                # Obtén el nombre del equipo usando el team_id
                team_name = teams_df[teams_df['teamId'] == team_id]['club_name'].iloc[0]
                print(f'Nombre del equipo para team_id {team_id}: {team_name}')
                break
            else:
                print(f'El team_id {team_id} no está en la base o no es válido.Por favor, introduce un team_id válido para la competición seleccionada.')
        except ValueError:
            print('Por favor, introduce un valor válido.')

    # Llamamos la función para saber en qué partidos participó el equipo y enviamos los parámetros de df_matches_country y el team_id
    matches_df = get_team_matches_df(df_matches_country, team_id)

    # Utilizamos un bucle para asegurarnos de que se ingrese un match_id válido
    while True:
        try:
            match_id = int(input('Introduce match_id: '))
            if match_id in matches_df['matchId'].values:
                match_is_valid(match_id, matches_df, teams_df)
                break
            else:
                print(f"El match_id {match_id} no está en la base o no es válido.Por favor, introduce un match_id válido.")
        except ValueError:
            print('Por favor, introduce un valor válido.')

    return competition_id, team_id, match_id, country_name, matches_df

# Call for the desired competition
Only English first division available for the being time

In [16]:
# Returns matches and teams for the desired league
def league_data(country_name):
  query_cntry=f'''
    select *
    from `{project_id}.consultor_dt.matches_{country_name}`
    order by gameweek
  '''
  df_matches = pd.read_gbq(query_cntry, project_id)
  #querying teams by country
  l=df_matches['home_team'].unique()
  teams='('
  for i in l:
    teams=teams+str(i)+','
  teams=teams[:-1]+')'
  query_teams=f'''
    select *
    from `{project_id}.consultor_dt.teams`
    where teamId in {teams}
  '''
  df_teams = pd.read_gbq(query_teams, project_id)

  return df_matches, df_teams



In [17]:
# Returns the season events game by team
def team_events(country_name,team_id):
  #querying matches by country
  query_cntry=f'''
    select players.shortName,
    teams.officialName,
events_{country_name}.*,
  players.role,
  players.weight,
  players.height
  from `{project_id}.consultor_dt.events_{country_name}` as events_{country_name}
    left join `{project_id}.consultor_dt.players` as players on players.playerId=events_{country_name}.playerId
    left join `{project_id}.consultor_dt.teams` as teams on teams.teamId=events_{country_name}.teamId
    where events_{country_name}.teamId={team_id}
    order by events_{country_name}.matchId, events_{country_name}.matchPeriod, events_{country_name}.eventSec
  '''
  df_events = pd.read_gbq(query_cntry, project_id)
  return df_events

In [18]:
# Returns the season matches game by team
def matches_by_team(country_name):
  query_cntry=f'''
    select *
    from `{project_id}.consultor_dt.matches_{country_name}`
    order by gameweek
  '''
  df_matches = pd.read_gbq(query_cntry, project_id)
  #querying teams by country
  l=df_matches['home_team'].unique()
  teams='('
  for i in l:
    teams=teams+str(i)+','
  teams=teams[:-1]+')'
  query_teams=f'''
    select *
    from `{project_id}.consultor_dt.teams`
    where teamId in {teams}
  '''
  df_teams = pd.read_gbq(query_teams, project_id)
  # filtering by team
  team_id='_'
  while team_id.isdigit()==False:
    print('Introduce team_id')
    print('')
    team_id=input()
  df_matches=df_matches.loc[(df_matches['home_team']==int(team_id))|(df_matches['away_team']==int(team_id))]
  df_matches=df_matches.merge(df_teams,left_on='home_team',right_on='teamId').drop(columns=['time','roundId','competitionId','teamId','country','alpha3code', 'city','club_name','home_team']).rename(columns={'officialName':'home_team'})
  df_matches=df_matches.merge(df_teams,left_on='away_team',right_on='teamId').drop(columns=['teamId','country','alpha3code', 'city','club_name','away_team']).rename(columns={'officialName':'away_team'})
  df_matches=df_matches.merge(df_teams,left_on='winner',right_on='teamId',how='left').drop(columns=['teamId','country','alpha3code', 'city','club_name']).fillna(0)
  df_matches=df_matches[['matchId', 'home_team', 'away_team','gameweek', 'seasonId', 'date', 'officialName', 'venue','score_home', 'score_away', 'scoreET_home', 'scoreET_away','scoreP_home', 'scoreP_away']]

  return df_matches

In [19]:
def plot_attacks(competition_id, team1, match_id, country_name):
  df=team_events(country_name,team1)
  df=df[df['matchId']==int(match_id)]

  # Getting rid of unsuccesful events
  #df=df[df['tag_1800']!=1802]

  #declare an empty dataframe
  danger_passes = pd.DataFrame()
  shots=pd.DataFrame()
  danger_others=pd.DataFrame()
  fkg=pd.DataFrame()

  for i in df['matchPeriod'].unique():
    #### mask goals
    mask_shot=(df.eventName=='Shot')&(df.matchPeriod==i)&(df.teamId==team1)&(df['tag_1800']!=1802)

    ##### mask pass
    mask_pass=(df.eventName=='Pass')&(df.matchPeriod==i)&(df.teamId==team1)

    ##### shot times
    shots_by_period=df[mask_shot]
    shot_times=df[mask_shot]['eventSec']

    #### shot window
    shot_window=15
    shot_start = shot_times - shot_window

    ### pass_times
    passes_by_period=df[mask_pass]
    pass_times=df[mask_pass]['eventSec']
    pass_to_shot = pass_times.apply(lambda x: True in ((shot_start < x) & (x < shot_times)).unique()) ##pandas.Series.apply ----> Invoke function on values of Series.

    ### others
    mask_others=(df.eventName!='Shot')&(df.matchPeriod==i)&(df.eventName!='Pass')&(df.teamId==team1)
    others_by_period=df[mask_others]
    others_times=df[mask_others]['eventSec']
    others_b_shot= others_times.apply(lambda x: True in ((shot_start < x) & (x < shot_times)).unique())

    #keep only danger passes_by_period
    danger_passes_period = passes_by_period.loc[pass_to_shot]

    #concatenate dataframe with a previous one to keep danger passes_by_period from the whole match
    danger_passes = pd.concat([danger_passes, danger_passes_period], ignore_index = True)

    #concatenate dataframe with a previous one to keep shots from the whole match
    shots=pd.concat([shots, shots_by_period], ignore_index = True)

    #concatenate dataframe with a previous one to keep others from the whole match
    others=others_by_period.loc[others_b_shot]
    danger_others=pd.concat([danger_others, others], ignore_index = True)

    #### mask Free kick goal
    mask_fkg=(df.tag_100==101)&(df.matchPeriod==i)&(df.teamId==team1)&(df.eventName=='Free Kick')
    fkg_by_period=df[mask_fkg]

    #concatenate dataframe with a previous one to keep shots from the whole match
    fkg=pd.concat([fkg, fkg_by_period], ignore_index = True)
  free_kicks=danger_others[danger_others['eventName']=='Free Kick'].reset_index()
  duels=danger_others[danger_others.eventName=='Duel'].reset_index()

  #actions before a free kick goal
  actions=pd.DataFrame()
  for i in fkg.index:
    actions_before=df.loc[(i-6):(i-1)]
    actions=pd.concat([actions, actions_before], ignore_index = True)

  ##### mask pass
  mask_actions_pass=(df.eventName=='Pass')

  ## mask others actions
  mask_others_actions=df.eventName!='Pass'

  # passes before free kick
  actions_passes=actions[mask_actions_pass]

  #others before free kick
  others_actions=actions[mask_others_actions]



  #plot pitch
  pitch = Pitch(pitch_color='#22312b', line_color='#c7d5cc', goal_type='box', goal_alpha=.8,axis=True,label=True)
  fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                      endnote_height=0.04, title_space=0, endnote_space=0)
  fig.set_facecolor('#22312b')


  if fkg.position1_x.any() :       # df.any() Returns False unless there is at least one element within a series or along a Dataframe axis that is True or equivalent (e.g. non-zero or non-empty).

    # free kick goals location
    pitch.scatter(fkg.position1_x.astype(float)*1.2, fkg.position1_y.astype(float)*0.8, color = "white", ax=ax['pitch'])

    # free kicks players
    for i, label in enumerate(fkg.shortName):
      pitch.annotate(label, (fkg.position1_x[i]*1.2, fkg.position1_y[i]*0.8),va='top', ha='center', color='magenta', fontsize=7, ax=ax['pitch'])


  # pass player
  for i, label in enumerate(danger_passes.shortName):
    pitch.annotate(
      label, (danger_passes.position1_x[i]*1.2, danger_passes.position1_y[i]*0.8),
                    va='top', ha='center', color='red', fontsize=15, ax=ax['pitch']
  )

  # Shots player
  for i, label in enumerate(shots.shortName):
    pitch.annotate(
      label, (shots.position1_x[i]*1.2, shots.position1_y[i]*0.8),xytext =(shots.position1_x[i]*1.2+2,shots.position1_y[i]*0.8-1),
                    va='top', ha='center', color='red', fontsize=7, ax=ax['pitch']
    )

    #  Duels players
  for i, label in enumerate(duels.shortName):
    pitch.annotate(label,(duels.position1_x[i]*1.2, duels.position1_y[i]*0.8),va='top', ha='center', color='white', fontsize=7, ax=ax['pitch'])

    # Free kick players
  for i, label in enumerate(free_kicks.shortName):
    pitch.annotate(
      label, (free_kicks.position1_x[i]*1.2, free_kicks.position1_y[i]*0.8),xytext =(free_kicks.position1_x[i]*1.2+2,free_kicks.position1_y[i]*0.8-1),
                    va='top', ha='center', color='red', fontsize=7, ax=ax['pitch']
    )



  #scatter the location on the pitch
  pitch.scatter(danger_passes.position1_x*1.2, danger_passes.position1_y*0.8, s=100, color='blue', edgecolors='grey', linewidth=1, alpha=0.2, ax=ax["pitch"])

  #uncomment it to plot arrows
  pitch.arrows(danger_passes.position1_x.astype(float)*1.2, danger_passes.position1_y.astype(float)*0.8, danger_passes.position2_x.astype(float)*1.2, danger_passes.position2_y.astype(float)*0.8, color = "blue", ax=ax['pitch'])
  pitch.scatter(shots.position1_x.astype(float)*1.2, shots.position1_y.astype(float)*0.8, color = "red", ax=ax['pitch'])
  pitch.arrows(duels.position1_x.astype(float)*1.2, duels.position1_y.astype(float)*0.8, duels.position2_x.astype(float)*1.2, duels.position2_y.astype(float)*0.8, color = "green", ax=ax['pitch'])
  pitch.arrows(free_kicks.position1_x.astype(float)*1.2, free_kicks.position1_y.astype(float)*0.8, free_kicks.position2_x.astype(float)*1.2, free_kicks.position2_y.astype(float)*0.8, color = "black", ax=ax['pitch'])



  return plt.show()



In [20]:
def pass_nodes(competition_id, team_id, match_id, country_name):
  df=team_events(country_name,team_id)
  df=df[(df['matchId']==int(match_id))]#
  last_minute=round(((df[df['matchPeriod']=='2H']['eventSec'].max())+(df[df['matchPeriod']=='1H']['eventSec'].max()))/60)
  df.drop(columns=['tag_100', 'tag_200',
       'tag_300', 'tag_400', 'tag_500', 'tag_600', 'tag_700', 'tag_800',
       'tag_900', 'tag_1000', 'tag_1100', 'tag_1200', 'tag_1300', 'tag_1400',
       'tag_1600', 'tag_1700',  'tag_1900', 'tag_2000', 'tag_2100'], inplace=True)
  df.drop(columns=['matchId','eventId','subEventId','id'], inplace=True)


  # second half events times plus last first half event
  df.loc[df['matchPeriod']=='2H','eventSec']=df[df['matchPeriod']=='2H'][['eventSec']]+(df[df['matchPeriod']=='1H'][['eventSec']].max())
  df=df[(df.tag_1800==1801)]
  df=df[(df.eventName=='Pass')]


  #columns passer and pass receiver. later on df will be grouped by passer and receiver so the # of passes between two player can be counted
  df['passer']=df['shortName']
  df['pass_receiver']=df['shortName'].shift(-1)

  query=f'''
    SELECT substitutions.matchId,
    substitutions.PlayerIn,
    substitutions.PlayerOut,
    substitutions.teamId,
    substitutions.minute,
    players.shortName as player_in,
    players2.shortName as player_out
    FROM `{project_id}.consultor_dt.substitutions` as substitutions
    join `{project_id}.consultor_dt.players` as players on players.playerId=substitutions.playerIn
    join `{project_id}.consultor_dt.players` as players2 on players2.playerId=substitutions.playerOut
    where substitutions.matchId={match_id} and substitutions.teamId={team_id}
    order by substitutions.minute'''
  substitutions=pd.read_gbq(query,project_id)
  substitutions['next_sub_min']=substitutions['minute'].shift(-1).fillna(last_minute)

  # first substitution for watford
  agg_1H_time=(df[df['matchPeriod']=='1H']['eventSec'].max())-(45*60)
  if substitutions['minute'].min()<=45:
    first_sub=substitutions['minute'].min()*60
  else:
    first_sub=(substitutions['minute'].min()*60)+agg_1H_time

  #initial formation
  init_form=df[df['eventSec']<=first_sub]

  # avg location
  mean_loc=init_form.groupby('shortName').agg({'position1_x':['mean'],'position1_y':['mean','count']})
  mean_loc.columns=['x','y','passes']

  #connections
  pass_between=df.groupby(['passer','pass_receiver']).shortName.count().reset_index()
  pass_between.rename({'shortName':'connections'}, axis='columns', inplace=True)
  pass_between=pass_between.merge(mean_loc,left_on='passer', right_index=True)
  pass_between=pass_between.merge(mean_loc,left_on='pass_receiver', right_index=True, suffixes=['','_receiver'])

  MAX_LINE_WIDTH=13
  pass_between['width']=(pass_between.connections / pass_between.connections.max())*MAX_LINE_WIDTH

  MIN_SOLIDNESS=0.000001
  # coordinates for connection color on RGBA axis
  color=np.array(to_rgba('yellow'))

  # Creation of color column for each connection on pass_between data frame
  color=np.tile(color,(len(pass_between),1))
  c_solidness=(pass_between.connections / pass_between.connections.max())
  c_solidness=(c_solidness*(1-MIN_SOLIDNESS))+MIN_SOLIDNESS
  color[:,3]=c_solidness

  # node size as function of nb of passes made
  MAX_MARKER_SIZE=1200

  mean_loc['marker_size']=(mean_loc['passes']/mean_loc.passes.max())*MAX_MARKER_SIZE

  #plot pitch
  pitch = Pitch(  pitch_color='#22312b', line_color='#c7d5cc', goal_type='box', goal_alpha=.8,axis=True,label=True)
  fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                      endnote_height=0.04, title_space=0, endnote_space=0)
  fig.set_facecolor('#22312b')
  arrows=pitch.arrows(pass_between.x.astype(float)*1.2,pass_between.y.astype(float)*0.8,pass_between.x_receiver.astype(float)*1.2,pass_between.y_receiver.astype(float)*0.8,ax=ax['pitch'],color=color,zorder=0.99)
  nodes=pitch.scatter(mean_loc.x.astype(float)*1.2,mean_loc.y.astype(float)*0.8, ax=ax['pitch'],
                    color='white',ec='black',s=mean_loc.marker_size )
  for i, row in mean_loc[['x','y']].iterrows():
    pitch.annotate(
        i,xy=(row.x*1.2-4,row.y*0.8-4), c='black',va='center',ha='center',size=8,
        weight='bold', ax=ax['pitch'],zorder=1,bbox=dict(boxstyle='round,pad=0.3', edgecolor='red', facecolor='red')
    )


  for i, row in substitutions.iterrows():
    #formation after substitution
    if row['minute']>=45:
      form=df[(df['eventSec']>(df[df['shortName']==row['player_out']]['eventSec'].max()))&(df['eventSec']<(((row['next_sub_min'])*60)))&(df.matchPeriod=='2H')]
    else:
      form=df[(df['eventSec']>(df[df['shortName']==row['player_out']]['eventSec'].max()))&(df['eventSec']<((row['next_sub_min'])*60))&(df.matchPeriod=='1H')]
    #print(df[df['shortName']==row['player_in']]['eventSec'].min())

      # avg location
    mean_loc=form.groupby('shortName').agg({'position1_x':['mean'],'position1_y':['mean','count']})
    mean_loc.columns=['x','y','passes']

    #connections
    pass_between=df.groupby(['passer','pass_receiver']).shortName.count().reset_index()
    pass_between.rename({'shortName':'connections'}, axis='columns', inplace=True)
    pass_between=pass_between.merge(mean_loc,left_on='passer', right_index=True)
    pass_between=pass_between.merge(mean_loc,left_on='pass_receiver', right_index=True, suffixes=['','_receiver'])


    MAX_LINE_WIDTH=13
    pass_between['width']=(pass_between.connections / pass_between.connections.max())*MAX_LINE_WIDTH

    MIN_SOLIDNESS=0.000001
    # coordinates for connection color on RGBA axis
    color=np.array(to_rgba('yellow'))

    # Creation of color column for each connection on pass_between data frame
    color=np.tile(color,(len(pass_between),1))
    c_solidness=(pass_between.connections / pass_between.connections.max())
    c_solidness=(c_solidness*(1-MIN_SOLIDNESS))+MIN_SOLIDNESS
    color[:,3]=c_solidness

    # node size as function of nb of passes made
    MAX_MARKER_SIZE=1200

    mean_loc['marker_size']=(mean_loc['passes']/mean_loc.passes.max())*MAX_MARKER_SIZE

    #plot pitch
    pitch = Pitch(  pitch_color='#22312b', line_color='#c7d5cc', goal_type='box', goal_alpha=.8,axis=True,label=True)
    fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                        endnote_height=0.04, title_space=0, endnote_space=0)
    fig.set_facecolor('#22312b')
    arrows=pitch.arrows(pass_between.x.astype(float)*1.2,pass_between.y.astype(float)*0.8,pass_between.x_receiver.astype(float)*1.2,pass_between.y_receiver.astype(float)*0.8,ax=ax['pitch'],color=color,zorder=0.99)
    nodes=pitch.scatter(mean_loc.x.astype(float)*1.2,mean_loc.y.astype(float)*0.8, ax=ax['pitch'],
                      color='white',ec='black',s=mean_loc.marker_size )
    for i, row in mean_loc[['x','y']].iterrows():
      pitch.annotate(
          i,xy=(row.x*1.2-4,row.y*0.8-4), c='black',va='center',ha='center',size=8,
          weight='bold', ax=ax['pitch'],zorder=1,bbox=dict(boxstyle='round,pad=0.3', edgecolor='red', facecolor='red')
      )


  print(substitutions)

  return plt.show()


# league_data(): Returns matches and teams for the desired league
# team_events(): Returns the season events game by team
# matches_by_team(): Returns the season matches game by team
# plot_attacks(): Returns a plot for the last passes 15 seconds before a succesful shot
# pass_nodes(teamId) Returns the node passes between substitutions #**the team id needs to be provided as argument**
